In [ ]:
import numpy as np
import pandas as pd
import re
import requests
import holoviews as hv
import pint
from cytoolz import dissoc

In [ ]:
%load_ext autoreload
%autoreload 2
import simulation

In [ ]:
# u = simulation.ureg
class ObjProxy(object):
    def __init__(self, module_name, attr_name):
        self.__module_name = module_name
        self.__attr_name = attr_name

    def __getattr__(self, name):
        return getattr(getattr(globals()[self.__module_name], self.__attr_name), name)


u = ObjProxy("simulation", "ureg")

In [ ]:
hv.extension("bokeh")

# Spectral response

In [ ]:
res[0]["spectra"][0].keys()

In [ ]:
def read_spectrum(id_):
    df = pd.read_csv(
        f"https://www.fpbase.org/spectra_csv/?q={id_}", index_col="wavelength"
    ).squeeze()
    # df.columns = [c.split(" ")[-1] for c in df.columns]
    return df

def get_fpbase_spectra():
    fpbase_entries = requests.get("https://www.fpbase.org/api/proteins/spectra/").json()
    fps = {}
    for fpbase_entry in fpbase_entries:
        fp = {}
        fp[]
        fp["spectra"] = {re.sub(r"^default_", "", s["state"]): s for s in fpbase_entry["spectra"]}
        res[fp["name"]] = fp
    return fps

In [ ]:
%%time
sources = {"SOLIS-3C": "7016", "SOLIS-565C": "7004"}

dichroics = {
    "Chroma ZT514rdc": "560",
    "Chroma ZT532rdc": "645",
    "Chroma T550lpxr": "803",
    "Chroma T556lpxr": "658",
    "Chroma ZT561rdc": "523",
    "Chroma ZT561rdc-xr": "708",
    "Chroma ZT568rdc": "604",
    "Chroma T570lpxr": "427",
    "Chroma T590lpxr": "593",
    "Chroma ZT594rdc": "569",
    "Chroma T600lpxr": "609",
    "Chroma T610lpxr": "601",
    "Chroma ZT633rdc": "668",
    "Chroma T635lpxr": "616",
    "Chroma ZT640rdc": "439",
}

longpass_filters = {
    "Chroma ET520LP": "760",
    "Chroma ET525lp": "530",
    "Chroma ET542lp": "461",
    "Chroma ET570lp": "487",
    "Chroma ET575lp": "620",
    "Chroma ET590lp": "805",
    "Chroma ET610lp": "350",
    "Chroma RET638lp": "2855",
    "Chroma ET655lp": "683",
    "Chroma ET665lp": "573",
}

sources = {name: read_spectrum(q) for name, q in sources.items()}
dichroics = {name: read_spectrum(q) for name, q in dichroics.items()}
longpass_filters = {name: read_spectrum(q) for name, q in longpass_filters.items()}

In [ ]:
%%time
fps = get_fpbase_spectra()

## Filters

In [ ]:
def combine_filter(dc, lp, threshold=0.9, max_wavelength=700):
    dc = dc.loc[:max_wavelength]
    lp = lp.loc[:max_wavelength]
    log_dc = np.log10(dc)
    log_lp = np.log10(lp)
    # highest wavelength that gives >90% excitation (dichroic reflectance)
    ex_cutoff = (dc > 1 - threshold)[::-1].idxmin()
    # highest wavelength that gives <90% transmission (dichroic+lp)
    em_cutoff = (log_dc + log_lp > np.log10(threshold)).idxmax()
    # gap between those wavelengths
    gap = em_cutoff - ex_cutoff
    # highest OD below excitation cutoff
    worst_rejection = (log_dc + log_lp).loc[:ex_cutoff].max()
    return pd.Series(
        {
            "ex_cutoff": ex_cutoff,
            "em_cutoff": em_cutoff,
            "gap": gap,
            "worst_rejection": worst_rejection,
        }
    )


filter_combinations = pd.concat(
    {
        dc_name: pd.concat(
            {
                lp_name: combine_filter(dc_spectrum, lp_spectrum)
                for lp_name, lp_spectrum in longpass_filters.items()
            },
            axis=1,
        ).T
        for dc_name, dc_spectrum in dichroics.items()
    }
)

In [ ]:
acceptable_filter_combinations = filter_combinations[
    (filter_combinations["gap"] < 30) & (filter_combinations["worst_rejection"] < -5.5)
]

In [ ]:
# for each dichroic, pick the longpass that minimizes the gap
selected_filter_combinations = (
    acceptable_filter_combinations.groupby(level=0)
    .apply(lambda x: x.loc[[x["gap"].idxmin()]])
    .droplevel(0)
    .index.values
)

## FPs

In [ ]:
res[0]["spectra"][0]

In [ ]:
[r["name"] for r in res]

In [ ]:
len(res)

In [ ]:
spectra_urls = {
    "mCherry": "https://www.fpbase.org/spectra_csv/?q=79,80,158",
    "mCherry2": "https://www.fpbase.org/spectra_csv/?q=1451,1450",
}
spectra = {
    name: pd.read_csv(url)
    .rename(columns={f"{name} {kind}": kind for kind in ("ex", "em", "2p")})
    .set_index("wavelength")
    for name, url in spectra_urls.items()
}

In [ ]:
bins = pd.interval_range(300, 1100, freq=5)

In [ ]:
binned_spectra = {
    name: bin_spectrum(spectrum, bins) for name, spectrum in spectra.items()
}

In [ ]:
(
    hv.Curve(spectra["mCherry"]["em"].dropna())
    * hv.Curve(spectra["mCherry2"]["em"].dropna())
).opts(width=800)

In [ ]:
(
    hv.Curve(binned_spectra["mCherry"]["em"].dropna())
    * hv.Curve(binned_spectra["mCherry2"]["em"].dropna())
).opts(width=800)